# Simple Pig-Latin Translator

## Task Eavesdropper

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_agents.message_queues.rabbitmq import RabbitMQMessageQueue
from multi_agent_system.utils import load_from_env

message_queue_host = load_from_env("RABBITMQ_HOST")
message_queue_port = load_from_env("RABBITMQ_NODE_PORT")
message_queue_username = load_from_env("RABBITMQ_DEFAULT_USER")
message_queue_password = load_from_env("RABBITMQ_DEFAULT_PASS")

message_queue = RabbitMQMessageQueue(
    url=f"amqp://{message_queue_username}:{message_queue_password}@{message_queue_host}:{message_queue_port}/"
)

In [ ]:
from llama_agents.types import TASK_CONSUMER_NAME
from llama_agents import QueueMessage, CallableMessageConsumer

task_messages = []


def handle_task_messages(message: QueueMessage) -> None:
    task_messages.append(message)


task_message_eavesdropper = CallableMessageConsumer(
    handler=handle_task_messages, message_type=TASK_CONSUMER_NAME
)

In [ ]:
task_eavesdropper_start_consuming = await message_queue.register_consumer(
    task_message_eavesdropper
)

INFO:llama_agents.message_queues.rabbitmq - Registered consumer 8302bac3-48bb-46c6-be0d-d3eb9ccebb9d: task_eavesdropper


In [ ]:
import asyncio

_ = asyncio.create_task(task_eavesdropper_start_consuming())

In [ ]:
task_messages

[QueueMessage(id_='180bd000-66b0-4537-a6da-8f371ee4c06b', publisher_id='default', data={'input': "Please decode the following input: 'ellohay owhay reaay ouyay'", 'task_id': '0fb355e3-e9f7-4e8c-9720-8676cf0d74fb', 'state': {}, 'agent_id': None, 'original_type': 'remove_ay_agent'}, action=<ActionTypes.NEW_TASK: 'new_task'>, stats=QueueMessageStats(publish_time='2024-07-07 22:51:14', process_start_time=None, process_end_time=None), type='task_eavesdropper'),
 QueueMessage(id_='56e39708-9a34-4bf8-9e76-6669658080ff', publisher_id='default', data={'input': "The decoded input is: 'elloh owh rea ouy'", 'task_id': '0fb355e3-e9f7-4e8c-9720-8676cf0d74fb', 'state': {}, 'agent_id': None, 'original_type': 'correct_first_character_agent'}, action=<ActionTypes.NEW_TASK: 'new_task'>, stats=QueueMessageStats(publish_time='2024-07-07 22:51:15', process_start_time=None, process_end_time=None), type='task_eavesdropper'),
 QueueMessage(id_='24103546-ce50-4603-ba3d-f122ae586152', publisher_id='default', dat

## Send Task

In [ ]:
from llama_agents import LlamaAgentsClient

In [ ]:
client = LlamaAgentsClient("http://0.0.0.0:8001")
task_id = client.create_task(
    "Please decode the following input: 'ellohay owhay reaay ouyay'"
)

In [ ]:
task_result = client.get_task_result(task_id)
print(task_result.result)

The corrected input is: "hello how are you"
